In [9]:
import os
import torch
print("torch module path:", torch.__file__)
print(torch.cuda.is_available())
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score, log_loss

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader, random_split

torch module path: c:\Users\Daria\anaconda3\Lib\site-packages\torch\__init__.py
False


In [10]:

data = pd.read_csv('Data/Final_data/final_bipolar_one_scaled')

data = data.iloc[:, 1:]
data.head()

,821,328813,823,817,818,239935,28116,351091,28118,1161942,...,68274,40091,1393,28133,305719,183,58180,29346,349096,Bipolar
0,2.327985,-0.408457,-0.421953,-0.200617,0.101252,-0.213354,-0.519106,1.912711,-0.033040,-0.259886,...,-0.193753,-0.203639,-0.15353,-0.114452,-0.117709,-0.193258,-0.228951,-0.191338,-0.074053,0
1,0.278163,-0.159743,-0.465366,-0.312955,-0.501173,-0.384932,-0.248024,-0.641646,-0.660877,-0.263336,...,-0.193753,-0.203639,-0.15353,-0.114452,-0.117709,-0.193258,-0.228951,-0.191338,-0.074053,0
2,-0.541187,-0.244025,0.225158,-0.317246,-0.570925,-0.338027,-0.360202,-0.607301,0.716744,-0.263336,...,-0.193753,2.515196,-0.15353,-0.114452,-0.117709,-0.193258,-0.228951,1.269814,-0.074053,0
3,0.072909,-0.368226,0.951357,-0.298452,1.087678,1.210338,0.613616,-0.764068,0.292560,-0.263336,...,-0.193753,1.720728,-0.15353,0.469804,-0.117709,-0.193258,-0.228951,-0.191338,-0.074053,0
4,-0.787331,-0.475470,-0.571947,0.082964,-0.311148,2.516107,0.562476,-0.710442,-0.585640,-0.263336,...,-0.193753,-0.203639,-0.15353,-0.114452,-0.117709,-0.193258,-0.228951,-0.191338,-0.074053,1


In [11]:
class Model(nn.Module):
    def __init__(self, input_size, h1, h2, h3, output_size = 2):
        super(Model, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, h1),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(h2, h3),
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(h3, output_size) 
        )
    def forward(self, x):
        return self.net(x)


In [12]:
class SklearnWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, input_size=10, h1=32, h2=16, h3=16, lr=0.01, num_epochs=100):
        self.input_size = input_size
        self.h1 = h1
        self.h2 = h2
        self.h3 = h3
        self.lr = lr
        self.num_epochs = num_epochs
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None

    def fit(self, X, y):
        dataset = TensorDataset(X, y)
        
        val_size = int(0.2 * len(dataset))
        train_size = len(dataset) - val_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32)

        self.model = Model(self.input_size, self.h1, self.h2, self.h3).to(self.device)
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=1e-4)
        criterion = nn.CrossEntropyLoss()

        best_loss = float('inf')
        patience = 20
        wait = 0
        best_state = None

        for epoch in range(self.num_epochs):
            self.model.train()
            for xb, yb in train_loader:
                xb, yb = xb.to(self.device), yb.to(self.device)
                pred = self.model(xb)
                loss = criterion(pred, yb)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        
            self.model.eval()
            val_loss = 0
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb, yb = xb.to(self.device), yb.to(self.device)
                    pred = self.model(xb)
                    val_loss += criterion(pred, yb).item()
                    

            val_loss /= len(val_loader)

            if val_loss < best_loss:
                best_loss = val_loss
                best_state = self.model.state_dict()
                wait = 0
            else:
                wait += 1
                if wait >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

        #if best_state:
         #   self.model.load_state_dict(best_state)
        
        self.best_state_dict = best_state
   
        if self.best_state_dict is not None:
            self.model.load_state_dict(self.best_state_dict)

        return self


    def predict(self, X):
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X)
        return outputs.argmax(dim=1).cpu().numpy()


In [13]:
param_grid = {
    'h1': [32, 64, 96, 128, 160],
    'h2': [16, 32, 64, 96],
    'h3': [16, 32, 64, 96],
    'lr': [0.0001, 0.0005],
    'num_epochs': [40, 60, 100, 120, 160, 200]
}

In [14]:
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr')
    #'neg_log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True) 
    #check loss calc
}

In [15]:
X = data.drop('Bipolar', axis = 1)
y = data['Bipolar']

X = X.values
y = y.values

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#X_train = torch.FloatTensor(X_train)
#X_test = torch.FloatTensor(X_test)

#y_train = torch.LongTensor(y_train)
#y_test = torch.LongTensor(y_test)

In [16]:

nested_scores = []

outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, test_idx in outer_cv.split(X, y):
    X_train_cv = torch.FloatTensor(X[train_idx])
    y_train_cv = torch.LongTensor(y[train_idx])
    X_test_cv = torch.FloatTensor(X[test_idx])
    y_test_cv = y[test_idx]

    clf = GridSearchCV(SklearnWrapper(input_size=X.shape[1]), param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    clf.fit(X_train_cv, y_train_cv)

    preds = clf.predict(X_test_cv)
    acc = accuracy_score(y_test_cv, preds)
    nested_scores.append(acc)
    print(f"Fold Accuracy: {acc:.4f}")

print(f"Nested CV Mean Accuracy: {np.mean(nested_scores):.4f}")


Early stopping at epoch 60
Fold Accuracy: 0.8790
Early stopping at epoch 28
Fold Accuracy: 0.8618
Early stopping at epoch 31
Fold Accuracy: 0.8780
Early stopping at epoch 36
Fold Accuracy: 0.9024
Early stopping at epoch 118
Fold Accuracy: 0.8862
Nested CV Mean Accuracy: 0.8815


In [17]:
print(clf.best_estimator_.model)

Model(
  (net): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=64, out_features=16, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=16, out_features=64, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=64, out_features=2, bias=True)
  )
)


In [18]:
#m = clf.best_state_dict

In [19]:
best_model = clf.best_estimator_

In [20]:
correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test_cv):
    y_val = best_model.model.forward(data)

    print(f'{i+1}.)  {str(y_val)} \t {y_test_cv[i]} \t {y_val.argmax().item()}')

    # Correct or not
    if y_val.argmax().item() == y_test_cv[i]:
      correct +=1

print(f'We got {correct} correct!')
accuracy = correct / list(y_test_cv.shape)[0]
print(f'Accuracy is: {accuracy}')

1.)  tensor([ 4.2681, -4.2017]) 	 0 	 0
2.)  tensor([-4.1285,  2.8182]) 	 1 	 1
3.)  tensor([ 11.1472, -10.9548]) 	 0 	 0
4.)  tensor([-1.8201,  1.2317]) 	 0 	 1
5.)  tensor([-0.1242, -0.1200]) 	 1 	 1
6.)  tensor([-1.2562,  0.7832]) 	 1 	 1
7.)  tensor([ 2.6743, -2.6965]) 	 0 	 0
8.)  tensor([ 4.9630, -4.9511]) 	 0 	 0
9.)  tensor([-0.3346,  0.0124]) 	 0 	 1
10.)  tensor([ 4.1854, -4.2065]) 	 0 	 0
11.)  tensor([-5.4658,  3.7691]) 	 1 	 1
12.)  tensor([-4.4571,  3.0267]) 	 1 	 1
13.)  tensor([-0.2359, -0.1268]) 	 1 	 1
14.)  tensor([ 4.6048, -4.5871]) 	 0 	 0
15.)  tensor([ 5.0272, -4.9819]) 	 0 	 0
16.)  tensor([-0.0497, -0.4422]) 	 0 	 0
17.)  tensor([-4.7324,  3.2714]) 	 1 	 1
18.)  tensor([-3.4659,  2.3461]) 	 1 	 1
19.)  tensor([ 0.3117, -0.4737]) 	 1 	 0
20.)  tensor([-5.4169,  3.6789]) 	 1 	 1
21.)  tensor([ 3.2559, -3.2315]) 	 0 	 0
22.)  tensor([-4.9478,  3.3954]) 	 1 	 1
23.)  tensor([ 4.4376, -4.4333]) 	 0 	 0
24.)  tensor([ 5.2661, -5.2157]) 	 0 	 0
25.)  tensor([-0.3699, 

In [21]:
print(list(y_test.shape))

NameError: name 'y_test' is not defined

In [ ]:
#torch.save(best_model.model.state_dict(), 'Models/3_layer_nn_96_64_64_with_dropout')

In [ ]:
#model = Model(input_size=X_train.shape[1], h1=160, h2=16, h3=16, output_size=2)
#model.load_state_dict(torch.load('Models/3_layer_nn_160_16_16'))
#model.eval()

In [ ]:
'''correct = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_val = model.forward(data)

    print(f'{i+1}.)  {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')

    # Correct or not
    if y_val.argmax().item() == y_test[i]:
      correct +=1

print(f'We got {correct} correct!')'''

"correct = 0\nwith torch.no_grad():\n  for i, data in enumerate(X_test):\n    y_val = model.forward(data)\n\n    print(f'{i+1}.)  {str(y_val)} \t {y_test[i]} \t {y_val.argmax().item()}')\n\n    # Correct or not\n    if y_val.argmax().item() == y_test[i]:\n      correct +=1\n\nprint(f'We got {correct} correct!')"